In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import Button, IntSlider, interactive_output, HBox, Output, Box
from amplify import Solver, BinarySymbolGenerator, einsum
from amplify.client import FixstarsClient
from amplify.constraint import one_hot, penalty

client = FixstarsClient()

%matplotlib inline

# 巡回セールスマン問題

巡回セールスマン問題とは、セールスマンが各都市を1回ずつ通り出発地点に戻ってくる場合に、最も短いを経路を求める問題です。

下記を検討する場合に、しばしば巡回セールスマン問題に準ずる課題が現れます。

* 配送ルートの最適化
* 集積回路の配線
* 生産計画におけるリソース配置

次のデモでは、指定された都市数に対しランダムに配置した問題を QUBO 定式化を用いて Amplify Annealing Engine で実行します。  
「Run」ボタンをクリックすると出力された解が表示されます。

In [ ]:
def tsp_initialize_problem(nc: int = 32):
    def gen_random_tsp():
        # 座標
        locations = np.random.uniform(size=(ncity, 2))

        # 距離行列
        all_diffs = np.expand_dims(locations, axis=1) - np.expand_dims(
            locations, axis=0
        )
        distances = np.sqrt(np.sum(all_diffs**2, axis=-1))

        return locations, distances

    def show_plot(locs: np.ndarray):
        plt.figure(figsize=(7, 7))
        plt.xlabel("x")
        plt.ylabel("y")
        plt.title("path length: ?")
        plt.scatter(*zip(*locations))
        plt.show()

    global ncity
    global locations
    global distances
    ncity = nc
    locations, distances = gen_random_tsp()
    show_plot(locations)


def tsp_solve(*args):
    solver = Solver(client)
    if ncity > 32:
        solver.client.parameters.timeout = 1500 + int((5000 - 1500) * (ncity - 32) / 32)
        k = 0.3
    else:
        solver.client.parameters.timeout = 1500
        k = 0.5
    solver.client.parameters.num_unit_steps = 5

    gen = BinarySymbolGenerator()
    q = gen.array(ncity, ncity)

    # 変数テーブルの要素に値を設定 (回転対称性の除去)
    q[0, 0] = 1
    q[0, 1:] = 0
    q[1:, 0] = 0

    # 各行の非ゼロ最小値をリストで取得
    d_min = [d[np.nonzero(d)].min() for d in distances]

    # コスト関数の係数を改変し定数項を加算
    cost = einsum("ij,ni,nj->", (distances - d_min), q, q.roll(-1, axis=0)) + sum(d_min)

    # 各行の最小値を引いた上で全要素の最大値を取得
    d_max_all = np.amax(distances - d_min)

    # 行に対する制約
    row_constraints = [one_hot(q[i]) for i in range(ncity)]

    # 列に対する制約
    col_constraints = [one_hot(q[:, i]) for i in range(ncity)]

    # 順序に対する制約 (反転対称性の除去)
    pem_constraint = [
        penalty(q[ncity - 1, i] * q[1, j])
        for i in range(ncity)
        for j in range(i + 1, ncity)
    ]

    constraints = sum(row_constraints) + sum(col_constraints) + sum(pem_constraint)

    model = cost + constraints * d_max_all * k

    result = solver.solve(model)
    while len(result) == 0 and k <= 1.0:
        k += 0.1
        model = cost + constraints * d_max_all * k
        result = solver.solve(model)

    energy, values = result[0].energy, result[0].values
    q_values = q.decode(values)
    route = np.where(np.array(q_values) == 1)[1]

    def show_route(route: list, time):
        path_length = sum(
            [distances[route[i]][route[(i + 1) % ncity]] for i in range(ncity)]
        )

        x = [i[0] for i in locations]
        y = [i[1] for i in locations]
        plt.figure(figsize=(7, 7))
        plt.title(f"path length: {path_length:.4f}, annealing time: {time:.2f}ms")
        plt.xlabel("x")
        plt.ylabel("y")

        for i in range(ncity):
            r = route[i]
            n = route[(i + 1) % ncity]
            plt.plot([x[r], x[n]], [y[r], y[n]], "b-")
        plt.plot(x, y, "ro")
        plt.show()

        return path_length

    show_route(route, solver.client_result.timing.time_stamps[-1])

In [ ]:
tsp_slider = IntSlider(
    value=32,
    min=4,
    max=64,
    step=1,
    description="num of cities:",
    disabled=False,
    continuous_update=False,
    orientation="horizontal",
    readout=True,
    readout_format="d",
)
tsp_run_btn = Button(description="Run", button_style="", tooltip="Run", icon="check")
tst_result_out = Output()


def show_tsp_problem(nc):
    tsp_initialize_problem(nc)
    with tst_result_out:
        tst_result_out.clear_output()


def show_tsp_result(btn):
    with tst_result_out:
        tst_result_out.clear_output()
        tsp_solve()


tsp_problem_out = interactive_output(show_tsp_problem, {"nc": tsp_slider})
tsp_run_btn.on_click(show_tsp_result)
display(HBox([tsp_slider, tsp_run_btn]), HBox([tsp_problem_out, tst_result_out]))